In [1]:
# import needed
import pandas as pd # data processing
import pandas_profiling as pp
import numpy as np # linear algebra
import pprint #indet json 
import requests #make http requests
from qwikidata.sparql  import return_sparql_query_results #return sparql results

In [2]:
#open datasets
df_movie_title = pd.read_csv('savelist.csv')
#df_movie_id = pd.read_csv('savelist2.csv')
df_serie_title = pd.read_csv('series_director.csv')
df_serie_id = pd.read_csv('series.csv')

In [6]:
df_first = pd.read_csv('0-6000.csv')
df_first = df_first.loc[:, ~df_first.columns.str.contains('^Unnamed')]
df_first.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
df_first.head(5)
#df_first.info()
#print(len(df_first))

,title,distributor,id,rating score
0,Sankofa,no_data,tt0108041,8.2/10
1,The Starling,Netflix,tt5164438,4.2/10
2,Confessions of an Invisible Girl,Netflix,tt15204288,no_data
3,Intrusion,no_data,tt5563324,4.5/10
4,Avvai Shanmughi,no_data,tt0139872,no_data


In [7]:
df_last = pd.read_csv('9000_end.csv')
df_last = df_last.loc[:, ~df_last.columns.str.contains('^Unnamed')]
df_last.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
df_last.head(5)

#df_last.info()
#print(len(df_last))

,title,distributor,id,rating score
0,Unforgivable,no_data,tt13003692,no_data
1,Sucker Punch,iTunes,tt0978764,4.2/10
2,Sherni,no_data,tt0187060,no_data
3,Shaandaar,Star Studios,tt4007558,3.9/10
4,LOL,Netflix,tt1592873,3.7/10


In [8]:
df_but_last = pd.read_csv('6000-9000.csv')
df_but_last = df_but_last.loc[:, ~df_but_last.columns.str.contains('^Unnamed')]
df_but_last.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
df_but_last.head(5)
#df_but_last.info()
#print(len(df_but_last))

,title,distributor,id,rating score
0,Slappy and the Stinkers,TriStar Pictures,tt0120213,3.3/10
1,Sky Racket,Sam Katzman,tt0140551,no_data
2,Sitting on the Moon,Republic Pictures,tt0028260,no_data
3,Sita,no_data,tt9665402,no_data
4,Sing Sing Nights,no_data,tt0027003,no_data


In [9]:
df_movie_id = pd.concat([df_first, df_but_last, df_last],axis=0, join="outer", sort=False)
df_movie_id.head(5)

,title,distributor,id,rating score
0,Sankofa,no_data,tt0108041,8.2/10
1,The Starling,Netflix,tt5164438,4.2/10
2,Confessions of an Invisible Girl,Netflix,tt15204288,no_data
3,Intrusion,no_data,tt5563324,4.5/10
4,Avvai Shanmughi,no_data,tt0139872,no_data


# take care of multiple distributors/ratings

In [10]:
# check
pd.concat(g for _, g in df_movie_id.groupby("title") if len(g) > 1)

,title,distributor,id,rating score
596,#Home,Amazon Prime Video,tt10534500,no_data
1950,#Home,Amazon Prime Video,tt10534500,no_data
565,1 Night in San Diego,no_data,tt9765226,no_data
1919,1 Night in San Diego,no_data,tt9765226,no_data
472,10 Endrathukulla,Star Studios,tt5128266,no_data
...,...,...,...,...
3782,Zubaan,no_data,tt3776484,no_data
4492,Zubaan,no_data,tt3776484,no_data
5018,Zubaan,no_data,tt3776484,no_data
5544,Zubaan,no_data,tt3776484,no_data


In [11]:
df_movie_id_copy = df_movie_id.drop_duplicates(keep='first')

In [12]:
# check
pd.concat(g for _, g in df_movie_id_copy.groupby("title") if len(g) > 1)

,title,distributor,id,rating score
1401,10 Things I Hate About You,Disney+,tt0147800,69%
1511,10 Things I Hate About You,Netflix,tt0147800,69%
1258,10 to Midnight,The Cannon Group,tt0085121,40%
2612,10 to Midnight,Netflix,tt0085121,40%
772,A Chinese Ghost Story,no_data,tt1861421,no_data
...,...,...,...,...
6998,Zandalee,Netflix,tt0101004,4/10
3784,Zoe,Netflix,tt6010628,4.3/10
5546,Zoe,Amazon Studios,tt6010628,4.3/10
3780,Zombieland,InterCom,tt1156398,7.4/10


In [13]:
# check
df_movie_id_copy.head(7)
df_movie_id_copy.query('title=="10 Things I Hate About You"')

,title,distributor,id,rating score
1401,10 Things I Hate About You,Disney+,tt0147800,69%
1511,10 Things I Hate About You,Netflix,tt0147800,69%


In [16]:
df_movie_id_toupled = df_movie_id_copy.groupby(['title', 'id']).agg(tuple).reset_index()
df_movie_id_toupled.head(7)

,title,id,distributor,rating score
0,#Alive,tt10620868,"(Netflix,)","(88%,)"
1,#Home,tt10534500,"(Amazon Prime Video,)","(no_data,)"
2,#Roxy,tt5646082,"(iTunes,)","(no_data,)"
3,#realityhigh,tt6119504,"(Netflix,)","(40%,)"
4,'76,tt2385126,"(YouTube,)","(no_data,)"
5,1 Chance 2 Dance,tt2388856,"(Netflix,)","(no_data,)"
6,1 Mile to You,tt2184233,"(Gravitas Ventures,)","(no_data,)"


In [17]:
# check
df_movie_id_toupled.query('title=="10 Things I Hate About You"')	

,title,id,distributor,rating score
13,10 Things I Hate About You,tt0147800,"(Disney+, Netflix)","(69%, 69%)"


In [18]:
df_movie_id_clean_copy = df_movie_id_toupled.copy() #estraiamo i valori singoli dalle tuple e manteniamo un vlore solo se sono duplicati i rotten rates
for n_index, title in df_movie_id_clean_copy['title'].iteritems():
    distributor = df_movie_id_clean_copy["distributor"][n_index]
    rating = df_movie_id_clean_copy["rating score"][n_index]
    
    if len(distributor)== 1:
        df_movie_id_clean_copy.at[n_index,'distributor']=distributor[0]

    if len(rating) > 1:
        
        for i in rating:
            rating_check = set()
            rating_check.add(i)
            
            if len(rating_check) <= 1:
                df_movie_id_clean_copy.at[n_index,'rating score']=rating[0]
                

    if len(rating) == 1:
        
        df_movie_id_clean_copy.at[n_index,'rating score']=rating[0]

In [19]:
df_movie_id_clean_copy

,title,id,distributor,rating score
0,#Alive,tt10620868,Netflix,88%
1,#Home,tt10534500,Amazon Prime Video,no_data
2,#Roxy,tt5646082,iTunes,no_data
3,#realityhigh,tt6119504,Netflix,40%
4,'76,tt2385126,YouTube,no_data
...,...,...,...,...
6817,Zubaan,tt3776484,no_data,no_data
6818,Æon Flux,tt0402022,Paramount Pictures,3.5/10
6819,Çarsi Pazar,tt4324242,Netflix,no_data
6820,Çok Filim Hareketler Bunlar,tt1640202,Netflix,no_data


In [20]:
# check
df_movie_id_clean_copy.query('title=="10 Things I Hate About You"')	

,title,id,distributor,rating score
13,10 Things I Hate About You,tt0147800,"(Disney+, Netflix)",69%


# prepare autor/autor gender dataframe

In [ ]:
df_movie_title #autor/autor gender
df_movie_id_clean_copy #id/distributor/rottenscore

In [21]:
df_movie_title.head(7)

,Unnamed: 0,title,director,director gender
0,0,Sankofa,Haile Gerima,male
1,1,The Starling,Theodore Melfi,male
2,2,Confessions of an Invisible Girl,no_data,no_data
3,3,Intrusion,no_data,no_data
4,4,Avvai Shanmughi,K. S. Ravikumar,male
5,5,Jeans,Nicolette Krebitz,female
6,6,Minsara Kanavu,Rajiv Menon,male


In [23]:
df_movie_title = df_movie_title.loc[:, ~df_movie_title.columns.str.contains('^Unnamed')]
df_movie_title.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
df_movie_title.head(7)

,title,director,director gender
0,Sankofa,Haile Gerima,male
1,The Starling,Theodore Melfi,male
2,Confessions of an Invisible Girl,no_data,no_data
3,Intrusion,no_data,no_data
4,Avvai Shanmughi,K. S. Ravikumar,male
5,Jeans,Nicolette Krebitz,female
6,Minsara Kanavu,Rajiv Menon,male


In [28]:
df_movie_title_copy = df_movie_title.drop_duplicates(keep='first')
len(df_movie_title)

6835

In [30]:
# check
print(len(df_movie_title_copy))
print(len(df_movie_id_clean_copy))


6688
6822


# merge 


In [31]:
df_movie_directors_copy = df_movie_title_copy.copy()
df_movie_directors_copy["distributor"] = " "
df_movie_directors_copy["id"] = " "
df_movie_directors_copy["rating score"] = " "
df_movie_directors_copy.head()

,title,director,director gender,distributor,id,rating score
0,Sankofa,Haile Gerima,male,,,
1,The Starling,Theodore Melfi,male,,,
2,Confessions of an Invisible Girl,no_data,no_data,,,
3,Intrusion,no_data,no_data,,,
4,Avvai Shanmughi,K. S. Ravikumar,male,,,


In [35]:
#check
df_movie_id_clean_copy.head(4)

,title,id,distributor,rating score
0,#Alive,tt10620868,Netflix,88%
1,#Home,tt10534500,Amazon Prime Video,no_data
2,#Roxy,tt5646082,iTunes,no_data
3,#realityhigh,tt6119504,Netflix,40%


In [39]:
#check
small_df_movie_directors_copy = df_movie_directors_copy[0:500]
small_df_movie_id_clean_copy = df_movie_id_clean_copy[0:500]

In [41]:
#check
small_df_movie_id_clean_copy

,title,id,distributor,rating score
0,#Alive,tt10620868,Netflix,88%
1,#Home,tt10534500,Amazon Prime Video,no_data
2,#Roxy,tt5646082,iTunes,no_data
3,#realityhigh,tt6119504,Netflix,40%
4,'76,tt2385126,YouTube,no_data
...,...,...,...,...
495,Arjun Reddy,tt7294534,no_data,no_data
496,Arjun: The Warrior Prince,tt2404027,Netflix,5.7/10
497,Arlo the Alligator Boy,tt13454122,Netflix,7.4/10
498,Armageddon,tt0120591,Netflix,38%


In [68]:
for index_n, title in df_movie_directors_copy['title'].iteritems():
   
    for index_id, title_id in df_movie_id_clean_copy['title'].iteritems():
        
        distributor = df_movie_id_clean_copy["distributor"][index_id]
        id = df_movie_id_clean_copy["id"][index_id]
        rating = df_movie_id_clean_copy["rating score"][index_id]
        
        if title == title_id:
            
            df_movie_directors_copy.at[index_n,'distributor']=distributor
            df_movie_directors_copy.at[index_n,'id']=id
            df_movie_directors_copy.at[index_n,'rating score']=rating 

In [69]:
#check
df_movie_directors_copy.head(7)

,title,director,director gender,distributor,id,rating score
0,Sankofa,Haile Gerima,male,no_data,tt0108041,8.2/10
1,The Starling,Theodore Melfi,male,Netflix,tt5164438,4.2/10
2,Confessions of an Invisible Girl,no_data,no_data,Netflix,tt15204288,no_data
3,Intrusion,no_data,no_data,no_data,tt5563324,4.5/10
4,Avvai Shanmughi,K. S. Ravikumar,male,no_data,tt0139872,no_data
5,Jeans,Nicolette Krebitz,female,no_data,tt0279885,no_data
6,Minsara Kanavu,Rajiv Menon,male,AVM Productions,tt0282003,no_data


In [71]:
df_movie_directors_copy.to_csv("merged-movie.csv")

In [72]:
#check
len(pd.concat(g for _, g in df_movie_directors_copy.groupby("title") if len(g) > 1))

22

In [73]:
#check
pd.concat(g for _, g in df_movie_directors_copy.groupby("title") if len(g) > 1)

,title,director,director gender,distributor,id,rating score
2410,Abhinetri,Subodh Mukherjee,male,no_data,tt0178218,no_data
5692,Abhinetri,Satish Pradhan,male,no_data,tt0178218,no_data
322,Bliss,Abdullah Oğuz,male,Amazon Studios,tt10333426,4.9/10
6057,Bliss,Doris Dörrie,female,Amazon Studios,tt10333426,4.9/10
924,Boss,R. Raghuraj,male,Netflix,tt2571140,3.9/10
4464,Boss,V. N. Aditya,male,Netflix,tt2571140,3.9/10
6774,Jack,Francis Ford Coppola,male,"(Netflix, InterCom)",tt0116669,4.4/10
6827,Jack,Elisabeth Scharang,female,"(Netflix, InterCom)",tt0116669,4.4/10
1594,Lionheart,Genevieve Nnaji,female,YouTube,tt7707314,100%
5071,Lionheart,Sheldon Lettich,male,YouTube,tt7707314,100%
